In [2]:
import pandas as pd
import numpy as np

In [3]:
## Loading the datasets
Tier_1  = pd.read_pickle('./curated_datasets/Tier_1.pickle')
Tier_2  = pd.read_pickle('./curated_datasets/Tier_2.pickle')
# Removing any shared comments from politics (I think we can keep politics comments by the way and random sample the user to get what we need)
Tier_1=Tier_1[Tier_1['sub']!='politics']

In [1]:
Tier_1  = pd.read_pickle('./curated_datasets/Tier_1.pickle')
Tier_2  = pd.read_pickle('./curated_datasets/Tier_2.pickle')
# Removing any shared comments from politics (I think we can keep politics comments by the way and random sample the user to get what we need)
Tier_1=Tier_1[Tier_1['sub']!='politics']
# Min number of words in body (3 right now)
Tier_1=Tier_1[Tier_1['body'].apply(lambda x: len(x.split()))>2]
# Years that you want your tier in
Year=[2016]
Tier_1=Tier_1[Tier_1['Y'].isin(Year)]

# Some more filters on tiers if needed
Tier_1_10plus = Tier_1['author'].value_counts()
Tier_1_10plus = Tier_1['author'].value_counts().reset_index()
Tier_1_10plus = Tier_1_10plus[Tier_1_10plus['count']>=10]
Tier_1  = Tier_1.loc[Tier_1['author'].isin(Tier_1_10plus['author'].to_list())]


data = Tier_1

NameError: name 'pd' is not defined

In [13]:
data

,Index,id,author,created_utc,parent_id,link_id,body,Y,YM,sub,subreddit_id,Bias,Cred,submission_body
948718,948718,cyi64u7,ultimis,1451638295,t3_3yz5f1,3yz5f1,I read recently on this subject that in politi...,2016,2016-01,Conservative,t5_2qh6p,0.674677,0.384738,Donald Trump Is Smart To Remind Voters Of Clin...
948757,948757,cyi9cdm,Abakala,1451654458,t3_3z0cfe,3z0cfe,This is what I've been saying for a long time....,2016,2016-01,Conservative,t5_2qh6p,0.504332,0.588008,Krauthammer's Take: Trump Promising 'Success W...
948775,948775,cyiau4n,MiyegomboBayartsogt,1451660278,t3_3yzt2a,3yzt2a,"Seemingly without effort, Donald Trump's presi...",2016,2016-01,Conservative,t5_2qh6p,0.468367,0.503747,Black Power Extremist Louis Farrakhan Warns Hi...
948826,948826,cyigkcw,VirginWizard69,1451673375,t3_3z0z09,3z0z09,I loves me some Whittle. \n\n,2016,2016-01,Conservative,t5_2qh6p,0.706065,0.293663,WATCH: Bill Whittle’s last monologue of the ye...
948859,948859,cyik9i9,RPDBF,1451679904,t3_3z1gr7,3z1gr7,Well at least he moved on from Its because the...,2016,2016-01,Conservative,t5_2qh6p,-0.177375,0.915285,Chris Christie loses his marbles
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266462,266462,dbu3onz,Not_Cleaver,1483200431,t3_5la3pb,5la3pb,I'll be avoiding that like the plague. I'll su...,2016,2016-12,Republican,NaN,-0.291483,0.577107,"Trump Inauguration: Everything To Expect, From..."
266496,266496,dbua6de,Fubartender,1483209855,t3_5l65pw,5l65pw,Social conservatism is just political correctn...,2016,2016-12,Republican,NaN,-0.138467,0.910061,Retiring Rep. Richard Hanna: GOP too intoleran...
266516,266516,dbud2xd,Not_Cleaver,1483214033,t3_5lamoy,5lamoy,How very presidential./s,2016,2016-12,Republican,NaN,0.476541,0.570520,Trump wishes love and a happy New Year to his ...
266549,266549,dbulav7,kinkuagesimo,1483226649,t3_5lamoy,5lamoy,I remember i was supposed to feel outraged whe...,2016,2016-12,Republican,NaN,0.476541,0.570520,Trump wishes love and a happy New Year to his ...


NameError: name 'data' is not defined

In [4]:
labeled = pd.read_pickle('./labeled_no_politics/2016/Tier_1.pickle')
labeled

,Index,id,author,created_utc,parent_id,link_id,body,Y,YM,sub,subreddit_id,Bias,Cred,submission_body,stance
948718,948718,cyi64u7,ultimis,1451638295,t3_3yz5f1,3yz5f1,I read recently on this subject that in politi...,2016,2016-01,Conservative,t5_2qh6p,0.674677,0.384738,Donald Trump Is Smart To Remind Voters Of Clin...,]favor
948775,948775,cyiau4n,MiyegomboBayartsogt,1451660278,t3_3yzt2a,3yzt2a,"Seemingly without effort, Donald Trump's presi...",2016,2016-01,Conservative,t5_2qh6p,0.468367,0.503747,Black Power Extremist Louis Farrakhan Warns Hi...,]favor
948826,948826,cyigkcw,VirginWizard69,1451673375,t3_3z0z09,3z0z09,I loves me some Whittle. \n\n,2016,2016-01,Conservative,t5_2qh6p,0.706065,0.293663,WATCH: Bill Whittle’s last monologue of the ye...,]favor
948859,948859,cyik9i9,RPDBF,1451679904,t3_3z1gr7,3z1gr7,Well at least he moved on from Its because the...,2016,2016-01,Conservative,t5_2qh6p,-0.177375,0.915285,Chris Christie loses his marbles,]favor
948863,948863,cyikofp,caprimulgidae,1451680644,t3_3z0cfe,3z0cfe,And for months pundits like Krauthammer have b...,2016,2016-01,Conservative,t5_2qh6p,0.504332,0.588008,Krauthammer's Take: Trump Promising 'Success W...,]favor
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266456,266456,dbu36hw,ActusPurus,1483199581,t3_5l65pw,5l65pw,Both parties have become more intolerant and e...,2016,2016-12,Republican,NaN,-0.138467,0.910061,Retiring Rep. Richard Hanna: GOP too intoleran...,]favor
266462,266462,dbu3onz,Not_Cleaver,1483200431,t3_5la3pb,5la3pb,I'll be avoiding that like the plague. I'll su...,2016,2016-12,Republican,NaN,-0.291483,0.577107,"Trump Inauguration: Everything To Expect, From...",]favor
266516,266516,dbud2xd,Not_Cleaver,1483214033,t3_5lamoy,5lamoy,How very presidential./s,2016,2016-12,Republican,NaN,0.476541,0.570520,Trump wishes love and a happy New Year to his ...,]favor
266549,266549,dbulav7,kinkuagesimo,1483226649,t3_5lamoy,5lamoy,I remember i was supposed to feel outraged whe...,2016,2016-12,Republican,NaN,0.476541,0.570520,Trump wishes love and a happy New Year to his ...,]against


In [10]:
labeled[labeled['stance'] == ']favor']


,Index,id,author,created_utc,parent_id,link_id,body,Y,YM,sub,subreddit_id,Bias,Cred,submission_body,stance
948718,948718,cyi64u7,ultimis,1451638295,t3_3yz5f1,3yz5f1,I read recently on this subject that in politi...,2016,2016-01,Conservative,t5_2qh6p,0.674677,0.384738,Donald Trump Is Smart To Remind Voters Of Clin...,]favor
948775,948775,cyiau4n,MiyegomboBayartsogt,1451660278,t3_3yzt2a,3yzt2a,"Seemingly without effort, Donald Trump's presi...",2016,2016-01,Conservative,t5_2qh6p,0.468367,0.503747,Black Power Extremist Louis Farrakhan Warns Hi...,]favor
948826,948826,cyigkcw,VirginWizard69,1451673375,t3_3z0z09,3z0z09,I loves me some Whittle. \n\n,2016,2016-01,Conservative,t5_2qh6p,0.706065,0.293663,WATCH: Bill Whittle’s last monologue of the ye...,]favor
948859,948859,cyik9i9,RPDBF,1451679904,t3_3z1gr7,3z1gr7,Well at least he moved on from Its because the...,2016,2016-01,Conservative,t5_2qh6p,-0.177375,0.915285,Chris Christie loses his marbles,]favor
948863,948863,cyikofp,caprimulgidae,1451680644,t3_3z0cfe,3z0cfe,And for months pundits like Krauthammer have b...,2016,2016-01,Conservative,t5_2qh6p,0.504332,0.588008,Krauthammer's Take: Trump Promising 'Success W...,]favor
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266448,266448,dbu20kp,kinkuagesimo,1483197584,t3_5l5l6q,5l5l6q,Putin must feel like a puppet master with a br...,2016,2016-12,Republican,NaN,-0.088115,0.867363,Trump praises 'very smart' Putin,]favor
266456,266456,dbu36hw,ActusPurus,1483199581,t3_5l65pw,5l65pw,Both parties have become more intolerant and e...,2016,2016-12,Republican,NaN,-0.138467,0.910061,Retiring Rep. Richard Hanna: GOP too intoleran...,]favor
266462,266462,dbu3onz,Not_Cleaver,1483200431,t3_5la3pb,5la3pb,I'll be avoiding that like the plague. I'll su...,2016,2016-12,Republican,NaN,-0.291483,0.577107,"Trump Inauguration: Everything To Expect, From...",]favor
266516,266516,dbud2xd,Not_Cleaver,1483214033,t3_5lamoy,5lamoy,How very presidential./s,2016,2016-12,Republican,NaN,0.476541,0.570520,Trump wishes love and a happy New Year to his ...,]favor


In [38]:
os.environ["CUDA_VISIBLE_DEVICES"] = "5"

print(f'running on cuda device {torch.cuda.current_device()}')

### Loading the Llama2 7b - chat model
base_model_name = "NousResearch/Llama-2-7b-chat-hf"

llama_tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
llama_tokenizer.pad_token = llama_tokenizer.eos_token
llama_tokenizer.padding_side = "right"

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False
)

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    quantization_config=quant_config,
    device_map='sequential'
    #max_memory=max_memory,
)
base_model.config.use_cache = False
base_model.config.pretraining_tp = 1

print(base_model.hf_device_map)

base_text_gen = pipeline(task="text-generation", model=base_model, tokenizer=llama_tokenizer, max_new_tokens=5)

query_head = "You are a helpful, respectful, and honest assistant that detects the stance of a comment with respect to its parent. Stance detection is the process of determining whether the author of a comment is in support of or against a given parent. You are provided with:\n post: the text you that is the root of discussion.\n parent:  the text which the comment is a reply towards.\n comment: text that you identify the stance from.\n\nYou will return the stance of the comment against the parent. Only return the stance against the parent and not the original post. Always answer from the possible options given below: \n support: The comment has a positive or supportive attitude towards the post, either explicitly or implicitly. \n against: The comment opposes or criticizes the post, either explicitly or implicitly. \n none: The comment is neutral or does not have a stance towards the post. \n unsure: It is not possible to make a decision based on the information at hand."
#query = "<SYS> query_head </SYS>" + "\n\n" + "post: " + row['submission_text'] + "\n" + "parent: " + row['body_parent'] + "\n" + "comment: " + row['body_child'] + "\n" + "stance: "
#query = "[INST] " + query + "[/INST]"
'''
query_tail="""post: Trump praises very smart Putin
comment: Maybe Trump is what democrats and republicans need to heal their relationship and start acting like adults again. We have a common cause: keeping this guy from doing permanent damage to a country we love.  The extremism that has entered US politics is stopping us from actually finding any common ground. \n\nIt's a big MAYBE, but it's possible.
stance:"""

query_head = query_head.strip()

output = base_text_gen(f"<s>[INST] {query_head + query_tail} [/INST]")
print(output[0]['generated_text'])
'''

NameError: name 'torch' is not defined